In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Masking, Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

2024-05-13 16:39:53.298915: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 16:39:53.340785: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 16:39:54.021760: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import numpy as np
import cv2  # OpenCVライブラリ
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import os
import sys
import pathlib
from pathlib import Path

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
import swallowing_recognition

In [4]:
current_file_path = pathlib.Path(os.getcwd())
parent_dir = current_file_path.parent

In [5]:
from swallowing_recognition import wavelet
from swallowing_recognition import audio
from swallowing_recognition.variable_data_set import VariableDataSet
from swallowing_recognition.mlp import MLP
from swallowing_recognition.cnn import CNN

In [6]:
directory_path = parent_dir / 'dataset'
   
train_voice_folder = directory_path / 'washino' / 'voice'
train_cough_folder = directory_path / 'washino' / 'cough'
train_swallowing_folder = directory_path / 'washino' / 'swallowing'   
train_neck_movement_folder = directory_path / 'washino' / 'neck_movement'
train_shoulder_movement_folder = directory_path / 'washino' / 'shoulder_movement'
train_arm_movement_folder = directory_path / 'washino' / 'arm_movement'   
train_hitting_folder = directory_path / 'washino' / 'hitting'   
train_mumble_folder = directory_path / 'washino' / 'mumble'   

test_voice_folder = directory_path / 'shibata' / 'voice'
test_cough_folder = directory_path / 'shibata' / 'cough'
test_swallowing_folder = directory_path / 'shibata' / 'swallowing'    
test_movement_folder = directory_path / 'shibata' / 'movement'    
test_hitting_folder = directory_path / 'shibata' / 'hitting'    

In [12]:
train_data = VariableDataSet(num_samples = 2000, scale = 222)
test_data = VariableDataSet(num_samples = 62, scale = 222)

In [13]:
train_data.folder_to_dataset(train_swallowing_folder, np.array(0), 0, signal_processing='fft')

In [14]:
train_data.folder_to_dataset(train_voice_folder, np.array(1), 700, signal_processing='fft')
train_data.folder_to_dataset(train_neck_movement_folder, np.array(1), 800, signal_processing='fft')
train_data.folder_to_dataset(train_shoulder_movement_folder, np.array(1), 900, signal_processing='fft')
train_data.folder_to_dataset(train_arm_movement_folder, np.array(1), 1000, signal_processing='fft')
train_data.folder_to_dataset(train_hitting_folder, np.array(1), 1100, signal_processing='fft')
train_data.folder_to_dataset(train_cough_folder, np.array(1), 1200, signal_processing='fft')
# train_data.folder_to_dataset(train_mumble_folder, np.array(1), 1300, signal_processing='fft')

857
[]
885
[]
908
[]
1065
[]
1076
[]
1103
[]
1283
[]


In [15]:
start_num = 1300

names = ['haruki', 'ezumi', 'hikaru', 'ibuki', 'kanata', 'kazuki', 'kishimoto', 'kosasa', 'mocchi', 'nakaryo', 'ruku', 'shibata', 'soichiro', 'tsuji']
for name in names:
    prediction_swallowing_folder = directory_path / name / 'swallowing'
    prediction_voice_folder = directory_path / name / 'voice'
    prediction_cough_folder = directory_path / name / 'cough'
    prediction_hitting_folder = directory_path / name / 'hitting'    
    prediction_movement_folder = directory_path / name / 'movement' 
    
    train_data.folder_to_dataset(prediction_swallowing_folder, np.array(0), start_num, signal_processing='fft')
    start_num += 10
    train_data.folder_to_dataset(prediction_voice_folder, np.array(1), start_num, signal_processing='fft')
    start_num += 10
    train_data.folder_to_dataset(prediction_cough_folder, np.array(1), start_num, signal_processing='fft')
    start_num += 10
    train_data.folder_to_dataset(prediction_hitting_folder, np.array(1), start_num, signal_processing='fft')
    start_num += 10
    train_data.folder_to_dataset(prediction_movement_folder, np.array(1), start_num, signal_processing='fft')
    start_num += 10
    print(start_num)

1350
1391
[]
1393
[]
1400
1450
1500
1550
1600
1650
1700
1709
[]
1750
1800
1850
1900
1950
1998
[]
2000


In [10]:
test_data.folder_to_dataset(test_swallowing_folder, np.array(0), 0, signal_processing='fft')
test_data.folder_to_dataset(test_cough_folder, np.array(1), 14, signal_processing='fft')
test_data.folder_to_dataset(test_voice_folder, np.array(1), 28, signal_processing='fft')
test_data.folder_to_dataset(test_movement_folder, np.array(1), 42, signal_processing='fft')
test_data.folder_to_dataset(test_hitting_folder, np.array(1), 52, signal_processing='fft')

In [16]:
print(type(train_data.data))
print((train_data.data).shape)
print((train_data.labels).shape)

<class 'numpy.ndarray'>
(2000, 222, 500)
(2000,)


In [17]:
from sklearn.model_selection import KFold
import numpy as np

# 5分割交差検証の設定
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# データを保持するためのリスト
train_data_splits = []
test_data_splits = []

# データのインデックスを分割
for train_index, test_index in kf.split(train_data.data):
    X_train, X_test = train_data.data[train_index], train_data.data[test_index]
    y_train, y_test = train_data.labels[train_index], train_data.labels[test_index]

    # 各フォールドのデータをリストに保存
    train_data_splits.append((X_train, y_train))
    test_data_splits.append((X_test, y_test))

# 結果の使用例
for i, (train, test) in enumerate(zip(train_data_splits, test_data_splits)):
    X_train, y_train = train
    X_test, y_test = test
    print(f"Fold {i+1}")
    print("Train data shape:", X_train.shape)
    print("Test data shape:", X_test.shape)
    print("Train labels shape:", y_train.shape)
    print("Test labels shape:", y_test.shape)
    print("Train number of 0s:", np.count_nonzero(y_train == 0))
    print("Train number of 1s:", np.count_nonzero(y_train == 1))
    print("Test number of 0s:", np.count_nonzero(y_test == 0))
    print("Test number of 1s:", np.count_nonzero(y_test == 1))

Fold 1
Train data shape: (1600, 222, 500)
Test data shape: (400, 222, 500)
Train labels shape: (1600,)
Test labels shape: (400,)
Train number of 0s: 676
Train number of 1s: 924
Test number of 0s: 174
Test number of 1s: 226
Fold 2
Train data shape: (1600, 222, 500)
Test data shape: (400, 222, 500)
Train labels shape: (1600,)
Test labels shape: (400,)
Train number of 0s: 673
Train number of 1s: 927
Test number of 0s: 177
Test number of 1s: 223
Fold 3
Train data shape: (1600, 222, 500)
Test data shape: (400, 222, 500)
Train labels shape: (1600,)
Test labels shape: (400,)
Train number of 0s: 666
Train number of 1s: 934
Test number of 0s: 184
Test number of 1s: 216
Fold 4
Train data shape: (1600, 222, 500)
Test data shape: (400, 222, 500)
Train labels shape: (1600,)
Test labels shape: (400,)
Train number of 0s: 694
Train number of 1s: 906
Test number of 0s: 156
Test number of 1s: 244
Fold 5
Train data shape: (1600, 222, 500)
Test data shape: (400, 222, 500)
Train labels shape: (1600,)
Test 

In [19]:
model = MLP(500, scale = 222)
model.training(train_data.data, train_data.labels, 80, 32)
model.evaluate(test_data.data, test_data.labels)
model.save('mlp_20240502_binary_ep80_bs32_v1.keras')

Epoch 1/80
40/40 [==============================] - 1s 10ms/step - loss: 0.5160 - accuracy: 0.8595 - val_loss: 0.4883 - val_accuracy: 0.7643
Epoch 2/80
40/40 [==============================] - 0s 6ms/step - loss: 0.2398 - accuracy: 0.9214 - val_loss: 1.0673 - val_accuracy: 0.7357
Epoch 3/80
40/40 [==============================] - 0s 6ms/step - loss: 0.1605 - accuracy: 0.9437 - val_loss: 1.2851 - val_accuracy: 0.7643
Epoch 4/80
40/40 [==============================] - 0s 5ms/step - loss: 0.1214 - accuracy: 0.9595 - val_loss: 1.7463 - val_accuracy: 0.7643
Epoch 5/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0943 - accuracy: 0.9690 - val_loss: 1.8525 - val_accuracy: 0.8000
Epoch 6/80
40/40 [==============================] - 0s 6ms/step - loss: 0.0800 - accuracy: 0.9762 - val_loss: 2.2319 - val_accuracy: 0.7786
Epoch 7/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0659 - accuracy: 0.9770 - val_loss: 2.5494 - val_accuracy: 0.7786
Epoch 8/80
40/40 [=

40/40 [==============================] - 0s 5ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 13.0523 - val_accuracy: 0.6786
Epoch 60/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 13.0667 - val_accuracy: 0.6786
Epoch 61/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 13.3853 - val_accuracy: 0.6786
Epoch 62/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 13.4217 - val_accuracy: 0.6786
Epoch 63/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 13.5212 - val_accuracy: 0.6786
Epoch 64/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 13.6355 - val_accuracy: 0.6786
Epoch 65/80
40/40 [==============================] - 0s 5ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 13.6263 - val_accuracy: 0.6786
Epoch 66/80
40/40 

In [15]:
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [22]:
model = CNN(scale = 222)

model_checkpoint = ModelCheckpoint(
    filepath='cnn_20240513_binary_model_4.keras',  # 保存するファイルのパス
    monitor='accuracy',        # 監視する値
    save_best_only=True,       # 最良のモデルのみを保存
    verbose=1,                 # 進行状況のメッセージ表示
    mode='max'                 # 監視
)

use_data = 4

X_train, y_train = train_data_splits[use_data] 
X_test, y_test = test_data_splits[use_data]

model.training(X_train, y_train, 150, 32, model_checkpoint = model_checkpoint)
model.evaluate(X_test, y_test)
model.save('cnn_20240513_binary_ep150_bs32_v4.keras')

Epoch 1/150
37/45 [=======================>......] - ETA: 0s - loss: 0.6418 - accuracy: 0.5971
Epoch 1: accuracy improved from -inf to 0.63194, saving model to cnn_20240513_binary_model_4.keras
45/45 [==============================] - 2s 12ms/step - loss: 0.6075 - accuracy: 0.6319 - val_loss: 0.4527 - val_accuracy: 0.8750
Epoch 2/150
42/45 [===========================>..] - ETA: 0s - loss: 0.3818 - accuracy: 0.8571
Epoch 2: accuracy improved from 0.63194 to 0.85417, saving model to cnn_20240513_binary_model_4.keras
45/45 [==============================] - 0s 8ms/step - loss: 0.3836 - accuracy: 0.8542 - val_loss: 0.3390 - val_accuracy: 0.8938
Epoch 3/150
42/45 [===========================>..] - ETA: 0s - loss: 0.2965 - accuracy: 0.8899
Epoch 3: accuracy improved from 0.85417 to 0.88889, saving model to cnn_20240513_binary_model_4.keras
45/45 [==============================] - 0s 8ms/step - loss: 0.2943 - accuracy: 0.8889 - val_loss: 0.3072 - val_accuracy: 0.8938
Epoch 4/150
45/45 [=====

Epoch 28/150
44/45 [============================>.] - ETA: 0s - loss: 0.0679 - accuracy: 0.9744
Epoch 28: accuracy did not improve from 0.98819
45/45 [==============================] - 0s 7ms/step - loss: 0.0672 - accuracy: 0.9750 - val_loss: 0.3916 - val_accuracy: 0.9000
Epoch 29/150
43/45 [===========================>..] - ETA: 0s - loss: 0.0500 - accuracy: 0.9833
Epoch 29: accuracy did not improve from 0.98819
45/45 [==============================] - 0s 7ms/step - loss: 0.0504 - accuracy: 0.9833 - val_loss: 0.5702 - val_accuracy: 0.9000
Epoch 30/150
45/45 [==============================] - ETA: 0s - loss: 0.0429 - accuracy: 0.9854
Epoch 30: accuracy did not improve from 0.98819
45/45 [==============================] - 0s 7ms/step - loss: 0.0429 - accuracy: 0.9854 - val_loss: 0.5163 - val_accuracy: 0.9000
Epoch 31/150
42/45 [===========================>..] - ETA: 0s - loss: 0.0443 - accuracy: 0.9866
Epoch 31: accuracy did not improve from 0.98819
45/45 [==============================

42/45 [===========================>..] - ETA: 0s - loss: 0.0306 - accuracy: 0.9888
Epoch 57: accuracy did not improve from 0.99583
45/45 [==============================] - 0s 8ms/step - loss: 0.0292 - accuracy: 0.9896 - val_loss: 0.5942 - val_accuracy: 0.9125
Epoch 58/150
39/45 [=========================>....] - ETA: 0s - loss: 0.0564 - accuracy: 0.9840
Epoch 58: accuracy did not improve from 0.99583
45/45 [==============================] - 0s 8ms/step - loss: 0.0575 - accuracy: 0.9833 - val_loss: 0.6710 - val_accuracy: 0.8500
Epoch 59/150
38/45 [========================>.....] - ETA: 0s - loss: 0.0358 - accuracy: 0.9901
Epoch 59: accuracy did not improve from 0.99583
45/45 [==============================] - 0s 8ms/step - loss: 0.0375 - accuracy: 0.9889 - val_loss: 0.5787 - val_accuracy: 0.8813
Epoch 60/150
42/45 [===========================>..] - ETA: 0s - loss: 0.0441 - accuracy: 0.9844
Epoch 60: accuracy did not improve from 0.99583
45/45 [==============================] - 0s 8ms/st

45/45 [==============================] - 0s 8ms/step - loss: 0.0080 - accuracy: 0.9951 - val_loss: 0.8086 - val_accuracy: 0.8875
Epoch 87/150
43/45 [===========================>..] - ETA: 0s - loss: 0.0078 - accuracy: 0.9978
Epoch 87: accuracy did not improve from 0.99861
45/45 [==============================] - 0s 7ms/step - loss: 0.0075 - accuracy: 0.9979 - val_loss: 0.9238 - val_accuracy: 0.8750
Epoch 88/150
44/45 [============================>.] - ETA: 0s - loss: 0.0066 - accuracy: 0.9979
Epoch 88: accuracy did not improve from 0.99861
45/45 [==============================] - 0s 7ms/step - loss: 0.0064 - accuracy: 0.9979 - val_loss: 0.9209 - val_accuracy: 0.8938
Epoch 89/150
43/45 [===========================>..] - ETA: 0s - loss: 0.0128 - accuracy: 0.9927
Epoch 89: accuracy did not improve from 0.99861
45/45 [==============================] - 0s 7ms/step - loss: 0.0129 - accuracy: 0.9924 - val_loss: 0.9331 - val_accuracy: 0.8813
Epoch 90/150
44/45 [============================>.] 

43/45 [===========================>..] - ETA: 0s - loss: 0.0080 - accuracy: 0.9971
Epoch 116: accuracy did not improve from 1.00000
45/45 [==============================] - 0s 7ms/step - loss: 0.0081 - accuracy: 0.9965 - val_loss: 0.8324 - val_accuracy: 0.9125
Epoch 117/150
44/45 [============================>.] - ETA: 0s - loss: 0.0126 - accuracy: 0.9964
Epoch 117: accuracy did not improve from 1.00000
45/45 [==============================] - 0s 7ms/step - loss: 0.0123 - accuracy: 0.9965 - val_loss: 0.6912 - val_accuracy: 0.9187
Epoch 118/150
45/45 [==============================] - ETA: 0s - loss: 0.0041 - accuracy: 0.9993
Epoch 118: accuracy did not improve from 1.00000
45/45 [==============================] - 0s 7ms/step - loss: 0.0041 - accuracy: 0.9993 - val_loss: 0.7092 - val_accuracy: 0.9187
Epoch 119/150
44/45 [============================>.] - ETA: 0s - loss: 0.0053 - accuracy: 0.9986
Epoch 119: accuracy did not improve from 1.00000
45/45 [==============================] - 0s

Epoch 146/150
45/45 [==============================] - ETA: 0s - loss: 0.0130 - accuracy: 0.9958
Epoch 146: accuracy did not improve from 1.00000
45/45 [==============================] - 0s 8ms/step - loss: 0.0130 - accuracy: 0.9958 - val_loss: 0.8696 - val_accuracy: 0.8875
Epoch 147/150
43/45 [===========================>..] - ETA: 0s - loss: 0.0165 - accuracy: 0.9964
Epoch 147: accuracy did not improve from 1.00000
45/45 [==============================] - 0s 7ms/step - loss: 0.0162 - accuracy: 0.9965 - val_loss: 0.7398 - val_accuracy: 0.8875
Epoch 148/150
43/45 [===========================>..] - ETA: 0s - loss: 0.0095 - accuracy: 0.9978
Epoch 148: accuracy did not improve from 1.00000
45/45 [==============================] - 0s 8ms/step - loss: 0.0094 - accuracy: 0.9979 - val_loss: 0.8840 - val_accuracy: 0.8750
Epoch 149/150
39/45 [=========================>....] - ETA: 0s - loss: 0.0080 - accuracy: 0.9960
Epoch 149: accuracy did not improve from 1.00000
45/45 [======================

In [ ]:
model.evaluate(test_data.data, test_data.labels)